# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [6]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from scipy import stats
from scipy.stats import ttest_ind
sns.set_style('darkgrid')
%matplotlib inline

df = pd.read_csv('homepage_actions.csv')
print(len(df))
df.head()

8188


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [23]:
df.sample(5)

,timestamp,id,group,action
7330,2017-01-06 18:14:25.775687,492186,experiment,view
2961,2016-11-06 08:33:51.867333,351179,experiment,view
4426,2016-11-27 06:33:17.730913,325412,experiment,click
5375,2016-12-10 16:47:36.140156,736704,experiment,click
7814,2017-01-13 09:17:38.588094,802456,control,click


In [24]:
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [25]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [2]:
clickers = set(df[df.action=='click']['id'].unique())
viewers = set(df[df.action=='view']['id'].unique())

In [3]:
print(len(clickers - viewers))
print(len(viewers - clickers))

0
4468


In [4]:
exp = set(df[df.group=='experiment']['id'].unique())
con = set(df[df.group=='control']['id'].unique())
print(len(exp&con))

0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [16]:
#Your code here
df['value'] = 1
df.head()

exclks = df.loc[(df.group == "experiment") & (df.action == 'click')]
conclks = df.loc[(df.group == "control") & (df.action == 'click')]

control = df[df.group=='control'].pivot(index='id', columns='action', values='value')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='value')
experiment = experiment.fillna(value=0)

stats.ttest_ind(control.click, experiment.click)

Ttest_indResult(statistic=-2.6195696844542202, pvalue=0.008825098914958293)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [20]:
#Your code here
rate = control.click.mean()
expected = rate * len(experiment)
expected

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [23]:
#Your code here
n = len(experiment)
p = rate
var = n * p * (1-p)
std = np.sqrt(var)

actual = experiment.click.sum()
z = (actual - expected)/std

z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [24]:
#Your code here
p_val = stats.norm.sf(z)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Looks like the p-value is lower, but both reject the null hypothesis.
So the experimental page is more effective.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.